In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import warnings
warnings.filterwarnings('ignore')

In [2]:
crsp = pd.read_csv(r'C:\Users\edgar\Downloads\crsp_top1000.csv', sep='\t')
FF = pd.read_csv(r'C:\Users\edgar\Downloads\FF.csv', sep='\t')
ratios = pd.read_csv(r'C:\Users\edgar\Downloads\financialratios.csv', sep='\t')
Ratios_Manual = pd.read_excel(r'C:\Users\edgar\Downloads\FinancialRatiosManual.xlsx')

C:\Users\edgar\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
FF['date'] = pd.to_datetime(FF['date'])
Ratios_Manual['Financial Ratio'] = Ratios_Manual['Financial Ratio'].str.replace('\n',' ')
crsp['date'] = pd.to_datetime(crsp['date'])
ratios['public_date'] = pd.to_datetime(ratios['public_date'])
df_all = pd.merge_asof(ratios.sort_values('public_date'), crsp.sort_values('date'), 
                           by='permno', 
                           left_on='public_date', 
                           right_on='date',
                           tolerance=pd.Timedelta(3, 'days'))
df_all = df_all.sort_values(by = ['permno', 'date'])

In [4]:
df_all

,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,...,date,prc,vol,ret,shrout,year,month,b_mkt,ivol,sic2
35792,10020,31dec1991,30sep1992,1993-01-31,21.557,0.64804,7.5821,NaN,NaN,13.942,...,1993-01-29,21.750000,5647.0,-.043956045,24493,1993,1,0.462073,0.020088,67
36641,10020,31dec1992,31dec1992,1993-02-28,19.962,0.57905,8.9928,NaN,NaN,16.250,...,1993-02-26,22.750000,2843.0,.045977011,24493,1993,2,0.505246,0.019377,67
37505,10020,31dec1992,31dec1992,1993-03-31,22.156,0.57905,8.9928,NaN,NaN,NaN,...,1993-03-31,25.250000,7312.0,.117362641,24493,1993,3,0.605275,0.016253,67
38436,10020,31dec1992,31dec1992,1993-04-30,20.840,0.57905,8.9928,NaN,NaN,NaN,...,1993-04-30,23.750000,4553.0,-.059405942,24493,1993,4,0.822465,0.015497,67
313238,10026,30sep2016,30sep2016,2017-01-31,35.467,0.30856,14.6000,31.344,31.499,31.499,...,2017-01-31,127.570000,13990.0,-.04391811,18709,2017,1,0.706220,0.009548,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378693,93436,31dec2021,30jun2022,2022-08-31,1003.500,0.05192,65.1320,29.414,32.889,33.166,...,2022-08-31,275.609985,7322218.0,-.072488703,3133470,2022,8,NaN,NaN,99
379378,93436,31dec2021,30jun2022,2022-09-30,973.360,0.05192,65.1320,28.308,31.653,31.919,...,2022-09-30,265.250000,12965500.0,-.037589297,3158000,2022,9,NaN,NaN,99
380559,93436,31dec2021,30jun2022,2022-10-31,834.920,0.05192,65.1320,24.284,27.153,27.381,...,2022-10-31,227.539993,17325849.0,-.142167792,3157752,2022,10,NaN,NaN,99
381306,93436,31dec2021,30sep2022,2022-11-30,412.500,0.04760,58.5920,53.834,59.969,60.217,...,2022-11-30,194.699997,18824244.0,-.144326255,3157752,2022,11,NaN,NaN,99


In [5]:
df_all_insample = df_all[df_all['public_date'] < '2017-01-01'].dropna(subset = ['prc'])
df_all_outsample = df_all[df_all['public_date'] >= '2017-01-01'].dropna(subset = ['prc'])

In [6]:
def Listret(fratio,df) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        test = df[df["date"] == Date[i]][['permno',fratio]].sort_values(fratio).dropna()
        testbottom20 = test[test[fratio] < test.quantile(.2)[fratio]]
        testtop20 = test[test[fratio] > test.quantile(.8)[fratio]]
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(list(testtop20['permno']))]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(list(testbottom20['permno']))]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [14]:
def Listret_Industry(fratio,df) :
    Date = list(df.sort_values('date')['date'].drop_duplicates())
    ListRet = []
    for i in range(len(Date)-1) :
        test = df[df["date"] == Date[i]][['permno','sic2',fratio]].sort_values(fratio).dropna()
        List_Industry = list(test['sic2'].drop_duplicates())
        List_Long = []
        List_Short = []
        for ind in List_Industry :
            testind = test[test['sic2'] == ind]
            testbottom20 = testind[testind[fratio] < testind.quantile(.2)[fratio]]
            List_Short += list(testbottom20['permno'].drop_duplicates())
            testtop20 = testind[testind[fratio] > testind.quantile(.8)[fratio]]
            List_Long += list(testtop20['permno'].drop_duplicates())
        df_all_test_top = df[df["date"] == Date[i+1]][df["permno"].isin(List_Long)]
        df_all_test_bottom = df[df["date"] == Date[i+1]][df["permno"].isin(List_Short)]
        ListRet.append((sum([float(x) for x in list(df_all_test_top['ret'])]) - sum([float(x) for x in list(df_all_test_bottom['ret'])]))/(len(df_all_test_top) + len(df_all_test_bottom))) 
    return(ListRet)

In [7]:
def Regress(df,ListRet) :
    df_regress = pd.DataFrame({'date' : list(df.sort_values('date')['date'].drop_duplicates())[1:], 'Return Portfolio' : ListRet}).merge(FF, how='left', on='date')
    df_regress['R_Rf'] = df_regress['Return Portfolio'] - df_regress['rf']  
    return(df_regress)

In [13]:
df1 = df_all_insample
df2 = df_all_outsample
fratio = 'roa'
ListRet1 = Listret(fratio,df1)
ListRet2 = Listret(fratio,df2)
df_reg = Regress(df2,ListRet2)
result = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
print('Return In Sample = ' + str(sum(ListRet1)))
print('Return Out Sample = ' + str(sum(ListRet2)))
print('Alpha = ' + str(result.params[0]))
print('Sharpe Ratio = ' + str(np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()))

Return In Sample = 0.714696180001335
Return Out Sample = 0.322232023959692
Alpha = 0.00027284655170227597
Sharpe Ratio = 0.6798369844932701


In [15]:
df1 = df_all_insample
df2 = df_all_outsample
fratio = 'roa'
ListRet1 = Listret_Industry(fratio,df1)
ListRet2 = Listret_Industry(fratio,df2)
df_reg = Regress(df2,ListRet2)
result = sm.ols(formula="R_Rf ~ mktrf+smb+hml+qual+umd", data=df_reg).fit()
print('Return In Sample = ' + str(sum(ListRet1)))
print('Return Out Sample = ' + str(sum(ListRet2)))
print('Alpha = ' + str(result.params[0]))
print('Sharpe Ratio = ' + str(np.sqrt(12)*df_reg['R_Rf'].mean()/df_reg['R_Rf'].std()))

Return In Sample = 0.4863461710590248
Return Out Sample = 0.11715726870976619
Alpha = -0.001080664365341768
Sharpe Ratio = 0.21918599878210254
